In [ ]:
%load_ext autoreload
%autoreload 2

In [2]:
from  ift6758.data.data_acquisition import Season


In [89]:
import pandas as pd
import numpy as np
def time_played(row):
    """
    return time in mins
    """
    # print(type(row['about.periodTime']))
    if row['result.event'] == 'GOAL' and row['about.period']>3:
        # return row['about.periodTime'] + row['about.periodTime']
        # .date.min()
        min_sec1 = row['about.periodTime'].split(':')
        return float(min_sec1[0]) + float(min_sec1[1])/60
    else:
        # return row['about.periodTime']
        # .date.min()
        min_sec1 = row['about.periodTime'].split(':')
        min_sec2 = row['about.periodTimeRemaining'].split(':')
        return float(min_sec1[0]) + float(min_sec2[0]) + (float(min_sec1[1]) +  float(min_sec2[1]))/60






In [87]:
# select_columns = ["result.event","gamePk","team.name","players","about.period","about.periodTime","about.periodType","about.periodTimeRemaining","coordinates.x","coordinates.y"]

def get_team_time(df_clean):
    sel_columns = ['gamePk','result.event','team.name','about.period','about.periodTime','about.periodTimeRemaining']
    df1 = df_clean[sel_columns].drop_duplicates() 
    df2 = df1.sort_values('about.periodTime', ascending=False).drop_duplicates(['gamePk','about.period'])
    df2['periodTimePlayed'] = df2.apply (lambda row: time_played(row), axis=1)
    df3 = df2.groupby('gamePk')['periodTimePlayed'].sum()
    df4 = df1[['gamePk','team.name']].drop_duplicates()
    df4['gameTime'] = df4.apply(lambda x: df3.get(key = x['gamePk']),axis=1)
    df5 = df4.groupby(['team.name'])['gameTime'].sum()
    total_time = df5.sum()
    return df5.to_dict(),total_time


    

time_game_dict = {}
for year in [2016,2017,2018,2019,2020]:
    season = Season(year,"../ift6758/data")
    df_clean = season.clean_data()
    time_game_dict[year] = {}
    time_game_dict[year]['team'],time_game_dict[year]['total_time'] = get_team_time(df_clean)
    time_game_dict[year]['teams_played'] = len(time_game_dict[year]['team'])





File already Exists, loading from ../ift6758/data/PICKLE//2016_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2017_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2018_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2019_clean.pkl
File already Exists, loading from ../ift6758/data/PICKLE//2020_clean.pkl


In [88]:
time_game_dict

{2016: {'team': {'Anaheim Ducks': 6591.1,
   'Arizona Coyotes': 5531.55,
   'Boston Bruins': 5834.033333333334,
   'Buffalo Sabres': 5230.733333333334,
   'Calgary Flames': 5692.283333333334,
   'Carolina Hurricanes': 5373.183333333333,
   'Chicago Blackhawks': 5659.216666666666,
   'Colorado Avalanche': 5268.216666666666,
   'Columbus Blue Jackets': 5827.866666666667,
   'Dallas Stars': 5309.55,
   'Detroit Red Wings': 5513.583333333333,
   'Edmonton Oilers': 6335.166666666667,
   'Florida Panthers': 5267.766666666666,
   'Los Angeles Kings': 5338.7,
   'Minnesota Wild': 5638.55,
   'Montréal Canadiens': 5815.25,
   'Nashville Predators': 6768.366666666667,
   'New Jersey Devils': 5418.366666666667,
   'New York Islanders': 5399.083333333333,
   'New York Rangers': 6193.533333333334,
   'Ottawa Senators': 6718.733333333334,
   'Philadelphia Flyers': 5450.4,
   'Pittsburgh Penguins': 7020.45,
   'San Jose Sharks': 5761.266666666666,
   'St. Louis Blues': 6114.35,
   'Tampa Bay Lightnin

In [21]:
import numpy as np
# def advanced_visualization(year=2018,team_name="Montréal Canadiens",im_size=(400,600)):
year=2018
team_name="Montréal Canadiens"
season = Season(year,"../ift6758/data")
df_clean = season.clean_data()
rink_size = (85,100)
num_teams = df_clean['team.name'].unique().shape[0]
total_matches = df_clean['gamePk'].unique().shape[0]


File already Exists, loading from ../ift6758/data/PICKLE//2018_clean.pkl


In [27]:
out = np.zeros(rink_size)
df_clean.head()
df_clean['x_bin']= np.floor(np.abs(df_clean['coordinates.x']))
df_clean['y_bin']= np.floor(42.5+df_clean['coordinates.y'])
num_games_by_team = df_clean[df_clean['team.name']==team_name]['gamePk'].unique().shape[0]
df_group1 = df_clean.groupby(['team.name','x_bin','y_bin'])['result.event'].count().reset_index()
# df_group2 = df_clean.groupby(['x_bin','y_bin'])['result.event'].count()
# df_group2 = df_group2/(time_game_dict[year]['total_time']*time_game_dict[year]['teams_played'])
# avg_shots_coordinate_dict = df_group2.to_dict()
df_group1['shots_average'] = df_group1['result.event']/num_games_by_team
df2 = df_group1.drop('result.event',axis=1).drop_duplicates()
df_group2 = df2.groupby(['x_bin','y_bin'])['shots_average'].sum()/num_teams
avg_shots_coordinate_dict = df_group2.to_dict()
df2['excess_shots'] = df2.apply(lambda row: row['shots_average'] - avg_shots_coordinate_dict[(row['x_bin'],row['y_bin'])],axis=1)
df3 = df2[df2['team.name']==team_name]




In [28]:
df3[(df3['x_bin']==75.0)&(df3['y_bin']==36.0)]

,team.name,x_bin,y_bin,shots_average,excess_shots
31267,Montréal Canadiens,75.0,36.0,0.044944,0.027929


In [30]:
def copyToOutput(row):
    global out
    out[int(row['y_bin']),int(row['x_bin'])]=row['excess_shots']
    return "copiedToOutput"
df3.apply(lambda row: copyToOutput(row),axis=1)

29665    copiedToOutput
29666    copiedToOutput
29667    copiedToOutput
29668    copiedToOutput
29669    copiedToOutput
              ...      
31703    copiedToOutput
31704    copiedToOutput
31705    copiedToOutput
31706    copiedToOutput
31707    copiedToOutput
Length: 2043, dtype: object

In [31]:
out[36][75]

0.027929373996789724